# Clustering of Aircraft Maintenance Data

This example shows the application of the clustering and tuning process with the example of aircraft maintenance difficulty reports. The test data is sourced from the FAA SDRS: https://sdrs.faa.gov/ and features reports of three different aircraft of the same type but from different airlines. This example will run the full clustering pipeline including clusterng tuning, some general and temporal anlysis. 

## Import Secton

In [ ]:
import pandas as pd
from importlib import resources
from clustering_pipeline import ClusteringPipeline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpec
from datetime import datetime



## Load Test Data

In [ ]:

PACKAGE_NAME = 'tactic.example_data'
RESOURCE_NAME = 'AircraftMaintenance.csv'


with resources.files(PACKAGE_NAME).joinpath(RESOURCE_NAME).open('r') as f:
    df = pd.read_csv(f, , encoding="latin1")

print(f"Loaded {len(df)} records")
print(f"Columns: {df.columns.tolist()}")

## Clustering
First Initalize a pipeline and load the data. The column relevant for clustering is the column, that includes the written discrepancy report. 

In [ ]:
pipeline = ClusteringPipeline(df, text_column='Discrepancy')



Run the full pipeline. Parameters for tuning are set for this example, however they can be modified as desired. The tsne tuning is False on default as it only serves the purpose of improving the visualization and is not improving clustering or clustering performance. 

In [ ]:
results = pipeline.full_pipeline(
        tune_hyperparameters=True,   # enables tuning instead of fixed params
        visualize=True,              # produce t-SNE visualization
        extract_keywords=True,       # extract keywords after clustering
        analyze_topics=True,         # run LDA topic modeling
        min_clusters=8,              # minimum cluster count requirement
        time_limit=300,              # tuning time in seconds
        standardize=True,            # scale data before clustering
        tune_tsne=False,             # skip t-SNE tuning
        random_state=33,             # ensure reproducibility
        num_topics=10,               # number of LDA topics
        passes=15,                   # number of LDA passes
        tf_top_n=5,                  # number of TF keywords per cluster
        yake_top_n=10,               # YAKE initial keywords
        yake_final_n=5,              # YAKE final filtered keywords
    )



## Analysis


This section of the notebook provides a comprehensive diagnostic report on the text clustering results generated by the pipeline.

***

### 1. Core Clustering Results & Statistics

The analysis establishes the quality and structure of the final clusters:

| Feature | Insight |
| :--- | :--- |
| **Total Clusters** | Reports the final number of distinct, meaningful clusters found. |
| **Noise Points** | Calculates the count and **noise ratio** (samples labeled as -1) to assess the effectiveness of HDBSCAN. |
| **Best Configuration** | Identifies the optimal **UMAP** (`n_neighbors`, `min_dist`) and **HDBSCAN** (`min_cluster_size`) parameters found during the tuning phase, along with the **Best Score** achieved. |
| **Cluster Size** | Prints the size distribution (number of points) for each final cluster. |
| **Visualizations** | Generates scatter plots of clusters in **t-SNE** and **UMAP** 2D space to visually confirm separation and density. |

***

### 2. Performance & Tuning Validation

The analysis validates the chosen solution by reviewing the hyperparameter search process:

* **Tuning Progress Plots:** Visualizes the evolution of key metrics like the **Davies-Bouldin Index (DBI)** (lower is better) and the **Calinski-Harabasz Index (CHI)** (higher is better) across all iterations.
* **Process Metrics:** Tracks the changes in the **Number of Clusters** and **Noise Ratio** during tuning, along with **Cumulative Execution Time**.
* **Performance Summary:** Provides a table summarizing the **best, worst, and average** values for all tracked clustering metrics.

***

### 3. Cluster Interpretation (Keywords)

The analysis interprets the thematic focus of each group:

* **Semantic Summary:** Prints a concise summary for each cluster, listing the top **YAKE Short keywords** to provide a quick, understandable label (e.g., Cluster 5: "brake, fluid, leak").
* **Detailed Analysis:** Provides a more extensive view by printing the top 8 YAKE Short keywords and contextualizing them with keywords from YAKE Long.

### Purpose and Overview 

The purpose of this analysis is to obtain an overview of the tuning process and how to interpet the outputs. In this case, the Davis Bouldin Index of the best iteration indicates a clear separation of the clusters while the Callinski-Harabsz Index is not the best one obtained. This indicates that there still is variance inside of the clusters. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.gridspec import GridSpec

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

def analyze_clustering_results(results):
    """
    Comprehensive analysis of clustering results
    """
    # Extract key components
    tuning_results = results['clustering']['tuning_results']
    clustering_results = results['clustering']['clustering_results']
    keywords_df = results['keywords']
    
    print("=" * 80)
    print("CLUSTERING ANALYSIS SUMMARY")
    print("=" * 80)
    
    # 1. Basic statistics
    clusters = clustering_results['clusters']
    unique_clusters = np.unique(clusters)
    n_clusters = len(unique_clusters) - (1 if -1 in unique_clusters else 0)
    n_noise = np.sum(clusters == -1)
    noise_ratio = n_noise / len(clusters)
    
    print(f"\n1. CLUSTERING OVERVIEW:")
    print(f"   • Total samples: {len(clusters)}")
    print(f"   • Number of clusters: {n_clusters}")
    print(f"   • Noise points: {n_noise} ({noise_ratio:.1%})")
    print(f"   • Cluster sizes: {np.bincount(clusters[clusters != -1] + 1)}")  # +1 to avoid negative indices
    
    # 2. Best parameters and performance
    best_params = tuning_results['best_params']
    best_score = tuning_results['best_score']
    
    print(f"\n2. BEST CONFIGURATION (Score: {best_score:.4f}):")
    print(f"   • UMAP: n_neighbors={best_params['umap']['n_neighbors']}, "
          f"min_dist={best_params['umap']['min_dist']}, "
          f"metric='{best_params['umap']['metric']}'")
    print(f"   • HDBSCAN: min_cluster_size={best_params['hdbscan']['min_cluster_size']}, "
          f"metric='{best_params['hdbscan']['metric']}', "
          f"method='{best_params['hdbscan']['cluster_selection_method']}'")
    
    # 3. Keyword analysis by cluster - USING YAKE SHORT
    print(f"\n3. CLUSTER KEYWORDS SUMMARY (YAKE Short):")
    for _, row in keywords_df.iterrows():
        cluster_id = row['cluster']
        if cluster_id != -1:  # Skip noise
            print(f"   • Cluster {cluster_id}:")
            yake_short_keywords = row['Yake Short'].split(', ')[:3]
            print(f"     - YAKE Short: {', '.join(yake_short_keywords)}")
    
    return {
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'noise_ratio': noise_ratio,
        'best_score': best_score,
        'best_params': best_params
    }

def plot_tuning_progress(tuning_results):
    """
    Create comprehensive plots showing tuning progress
    """
    history = tuning_results['history']
    
    # Extract metrics over iterations
    iterations = [entry['iteration'] for entry in history]
    davies_bouldin = [entry['metrics']['davies_bouldin'] for entry in history]
    calinski_harabasz = [entry['metrics']['calinski_harabasz'] for entry in history]
    n_clusters = [entry['metrics']['n_clusters'] for entry in history]
    noise_ratio = [entry['metrics']['noise_ratio'] for entry in history]
    time_elapsed = [entry['time_elapsed'] for entry in history]
    
    # Handle infinite values for plotting
    davies_bouldin_clean = [x if np.isfinite(x) else np.nan for x in davies_bouldin]
    calinski_harabasz_clean = [x if np.isfinite(x) else np.nan for x in calinski_harabasz]
    
    # Create subplots
    fig = plt.figure(figsize=(16, 12))
    gs = GridSpec(3, 2, figure=fig)
    
    # Plot 1: Davies-Bouldin Index (lower is better)
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(iterations, davies_bouldin_clean, 'o-', linewidth=2, markersize=4, alpha=0.7)
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Davies-Bouldin Index')
    ax1.set_title('Davies-Bouldin Index (Lower = Better)')
    ax1.grid(True, alpha=0.3)
    
    # Mark best iteration
    best_idx = davies_bouldin_clean.index(min([x for x in davies_bouldin_clean if not np.isnan(x)]))
    ax1.plot(iterations[best_idx], davies_bouldin_clean[best_idx], 'ro', markersize=8, label='Best')
    ax1.legend()
    
    # Plot 2: Calinski-Harabasz Index (higher is better)
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(iterations, calinski_harabasz_clean, 'o-', linewidth=2, markersize=4, alpha=0.7, color='green')
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Calinski-Harabasz Index')
    ax2.set_title('Calinski-Harabasz Index (Higher = Better)')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Number of clusters and noise ratio
    ax3 = fig.add_subplot(gs[1, 0])
    ax3_alt = ax3.twinx()
    
    line1 = ax3.plot(iterations, n_clusters, 'o-', linewidth=2, markersize=4, alpha=0.7, color='purple', label='Number of Clusters')
    line2 = ax3_alt.plot(iterations, noise_ratio, 's-', linewidth=2, markersize=4, alpha=0.7, color='orange', label='Noise Ratio')
    
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Number of Clusters', color='purple')
    ax3_alt.set_ylabel('Noise Ratio', color='orange')
    ax3.set_title('Cluster Count vs Noise Ratio')
    ax3.grid(True, alpha=0.3)
    
    # Combine legends
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax3.legend(lines, labels, loc='upper right')
    
    # Plot 4: Cumulative time
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.plot(iterations, time_elapsed, 'o-', linewidth=2, markersize=4, alpha=0.7, color='red')
    ax4.set_xlabel('Iteration')
    ax4.set_ylabel('Cumulative Time (seconds)')
    ax4.set_title('Cumulative Execution Time')
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: Parameter distributions
    ax5 = fig.add_subplot(gs[2, :])
    
    # Extract parameter values
    umap_nn = [entry['params']['umap']['n_neighbors'] for entry in history]
    umap_md = [entry['params']['umap']['min_dist'] for entry in history]
    hdbscan_mcs = [entry['params']['hdbscan']['min_cluster_size'] for entry in history]
    
    # Create parameter evolution plot
    param_data = pd.DataFrame({
        'Iteration': iterations,
        'UMAP n_neighbors': umap_nn,
        'UMAP min_dist': umap_md,
        'HDBSCAN min_cluster_size': hdbscan_mcs,
        'Davies-Bouldin': davies_bouldin_clean
    })
    
    # Normalize for plotting
    for col in ['UMAP n_neighbors', 'UMAP min_dist', 'HDBSCAN min_cluster_size']:
        param_data[col + '_norm'] = (param_data[col] - param_data[col].min()) / (param_data[col].max() - param_data[col].min())
    
    # Plot normalized parameters
    ax5.plot(iterations, param_data['UMAP n_neighbors_norm'], 'o-', label='UMAP n_neighbors', alpha=0.7)
    ax5.plot(iterations, param_data['UMAP min_dist_norm'], 's-', label='UMAP min_dist', alpha=0.7)
    ax5.plot(iterations, param_data['HDBSCAN min_cluster_size_norm'], '^-', label='HDBSCAN min_cluster_size', alpha=0.7)
    
    # Add Davies-Bouldin index (inverted and normalized for comparison)
    db_norm = 1 - (param_data['Davies-Bouldin'] - param_data['Davies-Bouldin'].min()) / (param_data['Davies-Bouldin'].max() - param_data['Davies-Bouldin'].min())
    ax5.plot(iterations, db_norm, 'd-', linewidth=2, label='Performance (1 - DB norm)', alpha=0.9, color='black')
    
    ax5.set_xlabel('Iteration')
    ax5.set_ylabel('Normalized Values')
    ax5.set_title('Parameter Evolution vs Performance')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig

def plot_cluster_visualizations(clustering_results, keywords_df):
    """
    Create visualizations of the clustering results
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Extract data
    tsne_embeddings = clustering_results['tsne_embeddings']
    clusters = clustering_results['clusters']
    umap_embeddings = clustering_results['umap_embeddings']
    
    # Plot 1: t-SNE visualization
    scatter1 = axes[0, 0].scatter(tsne_embeddings[:, 0], tsne_embeddings[:, 1], 
                                 c=clusters, cmap='tab20', alpha=0.7, s=30)
    axes[0, 0].set_xlabel('t-SNE Component 1')
    axes[0, 0].set_ylabel('t-SNE Component 2')
    axes[0, 0].set_title('t-SNE Projection of Clusters')
    plt.colorbar(scatter1, ax=axes[0, 0], label='Cluster ID')
    
    # Plot 2: UMAP first two components
    scatter2 = axes[0, 1].scatter(umap_embeddings[:, 0], umap_embeddings[:, 1], 
                                 c=clusters, cmap='tab20', alpha=0.7, s=30)
    axes[0, 1].set_xlabel('UMAP Component 1')
    axes[0, 1].set_ylabel('UMAP Component 2')
    axes[0, 1].set_title('UMAP Projection (Components 1-2)')
    plt.colorbar(scatter2, ax=axes[0, 1], label='Cluster ID')
    
    # Plot 3: Cluster size distribution
    cluster_counts = pd.Series(clusters).value_counts().sort_index()
    colors = [plt.cm.tab20(i % 20) for i in range(len(cluster_counts))]
    bars = axes[1, 0].bar(cluster_counts.index.astype(str), cluster_counts.values, color=colors, alpha=0.7)
    axes[1, 0].set_xlabel('Cluster ID')
    axes[1, 0].set_ylabel('Number of Points')
    axes[1, 0].set_title('Cluster Size Distribution')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                       f'{int(height)}', ha='center', va='bottom')
    
    # Plot 4: Top keywords by cluster - USING YAKE SHORT
    axes[1, 1].axis('off')
    cluster_info = []
    for _, row in keywords_df.iterrows():
        if row['cluster'] != -1:
            # Use YAKE Short keywords instead of TF-IDF
            top_keywords = ', '.join(row['Yake Short'].split(', ')[:5])
            cluster_info.append(f"Cluster {row['cluster']}: {top_keywords}")
    
    # Display top cluster keywords
    text_content = "\n".join(cluster_info[:8])  # Show first 8 clusters
    axes[1, 1].text(0.05, 0.95, "Top Cluster Keywords (YAKE Short):\n" + text_content, 
                   transform=axes[1, 1].transAxes, fontsize=10, 
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    return fig

def create_detailed_keyword_analysis(keywords_df):
    """
    Create a detailed analysis of YAKE Short keywords across clusters
    """
    print("\n" + "=" * 80)
    print("DETAILED YAKE SHORT KEYWORD ANALYSIS")
    print("=" * 80)
    
    # Filter out noise cluster
    meaningful_clusters = keywords_df[keywords_df['cluster'] != -1]
    
    for _, row in meaningful_clusters.iterrows():
        cluster_id = row['cluster']
        yake_short_keywords = row['Yake Short'].split(', ')
        
        print(f"\nCluster {cluster_id} - Top YAKE Short Keywords:")
        print("  " + ", ".join(yake_short_keywords[:8]))  # Show top 8 keywords
        
        # Compare with other methods for context
        yake_long_top = row['Yake Long'].split(', ')[:3]
        print(f"  Context (YAKE Long): {', '.join(yake_long_top)}")

def create_performance_summary(tuning_results):
    """
    Create a summary table of performance metrics
    """
    history = tuning_results['history']
    
    # Calculate summary statistics
    davies_bouldin_vals = [entry['metrics']['davies_bouldin'] for entry in history if np.isfinite(entry['metrics']['davies_bouldin'])]
    calinski_harabasz_vals = [entry['metrics']['calinski_harabasz'] for entry in history if np.isfinite(entry['metrics']['calinski_harabasz'])]
    n_clusters_vals = [entry['metrics']['n_clusters'] for entry in history]
    noise_ratios = [entry['metrics']['noise_ratio'] for entry in history]
    
    summary_df = pd.DataFrame({
        'Metric': ['Davies-Bouldin Index', 'Calinski-Harabasz Index', 'Number of Clusters', 'Noise Ratio'],
        'Best': [min(davies_bouldin_vals), max(calinski_harabasz_vals), 
                max(n_clusters_vals), min(noise_ratios)],
        'Worst': [max(davies_bouldin_vals), min(calinski_harabasz_vals), 
                 min(n_clusters_vals), max(noise_ratios)],
        'Average': [np.mean(davies_bouldin_vals), np.mean(calinski_harabasz_vals),
                   np.mean(n_clusters_vals), np.mean(noise_ratios)],
        'Std Dev': [np.std(davies_bouldin_vals), np.std(calinski_harabasz_vals),
                   np.std(n_clusters_vals), np.std(noise_ratios)]
    })
    
    print("\n4. PERFORMANCE SUMMARY ACROSS ALL ITERATIONS:")
    print(summary_df.to_string(index=False, float_format='%.3f'))
    
    return summary_df

# Main execution
if __name__ == "__main__":
    # Assuming your results are stored in a variable called 'results'
    # results = your_clustering_output
    
    # 1. Analyze and print summary
    analysis_summary = analyze_clustering_results(results)
    
    # 2. Create performance summary table
    performance_summary = create_performance_summary(results['clustering']['tuning_results'])
    
    # 3. Detailed YAKE Short keyword analysis
    create_detailed_keyword_analysis(results['keywords'])
    
    # 4. Plot tuning progress
    print("\n5. GENERATING TUNING PROGRESS PLOTS...")
    tuning_fig = plot_tuning_progress(results['clustering']['tuning_results'])
    
    # 5. Plot cluster visualizations
    print("\n6. GENERATING CLUSTER VISUALIZATIONS...")
    cluster_fig = plot_cluster_visualizations(
        results['clustering']['clustering_results'], 
        results['keywords']
    )


# Temporal Trend Analysis Summary

This section provides a deep dive into the **temporal trends** of the text data, examining how the identified clusters (themes/topics) evolve, emerge, persist, and fluctuate over time. The analysis relies on the **Submission Date** of each report.

***

## 1. Temporal Data Overview & Statistics

The analysis begins by prepping the date data and calculating fundamental temporal statistics:

* **Data Range:** Reports the minimum and maximum submission dates to establish the **time span** of the dataset.
* **Temporal Segmentation:** Segments the data into **Year** and **Month** for granular analysis.
* **Cluster Persistence Table:** Generates a statistical summary table for each cluster, detailing its:
    * **First and Last Year** of appearance (emergence and decline/persistence).
    * **Total Reports** and **Years Active**.
    * **Yearly Growth Rate** (from start to end year) to quantify the cluster's momentum.
* **Overall Trend Insights:** Calculates and prints high-level insights, including:
    * The **Year** with the **most and least cluster diversity** (number of unique clusters).
    * The **Overall Growth Rate** of report volume across the entire time frame.
    * **New/Emerging Clusters** identified in the most recent year compared to the previous year.
    * **Most Persistent Clusters** (those present in every year of the dataset).

***

## 2. Key Temporal Visualizations

A comprehensive set of four plots is generated to visualize cluster dynamics:

| Plot Type | X-Axis | Y-Axis | Insight Provided |
| :--- | :--- | :--- | :--- |
| **Stacked Bar Chart** | Year | % of Reports (Stacked by Cluster) | Shows the **changing dominance** or composition of clusters year-over-year. |
| **Yearly Trends Line Plot** | Year | Number of Reports | Tracks the absolute **growth/decline trajectory** of the top 6 largest clusters. |
| **Cluster Emergence Timeline** | Date | Cluster ID | Shows the **cumulative growth** of all reports and plots a point for the **first appearance** of each individual cluster. |
| **Monthly/Seasonal Patterns** | Month | % of Monthly Reports | Identifies **seasonal fluctuations** by showing the percentage contribution of the top clusters across the 12 months. |

***

## 3. Advanced Temporal Comparison

Two additional plots compare volume and diversity trends:

* **Volume vs. Diversity:** A combination plot showing **Total Report Volume** (Bar Chart) and **Cluster Diversity** (Line Plot) by year to assess if growth in volume leads to more diverse problems/themes.
* **Yearly Dominance:** A line plot tracking the **Percentage of Yearly Reports** for the top 3 clusters, clearly illustrating shifts in which theme is the most dominant in a given year.

***

## 4. Correlation Analysis

* **Yearly Frequency Correlation:** Calculates and plots a **Heatmap** of the **Pearson correlation** between the yearly frequency counts of all meaningful clusters.
* **Strong Correlation Output:** Prints a table listing pairs of clusters whose yearly frequencies are highly correlated ($|r| > 0.5$). This suggests that **these two themes tend to rise and fall together** over time.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime

def analyze_temporal_trends(results):
    """
    Analyze how clusters are distributed over time using submission dates.
    This function preprocesses the data for temporal analysis.
    """
    # Extract dataframe and clustering results
    df = results['dataframe']
    clusters = results['clustering']['clustering_results']['clusters']
    
    # Add clusters to dataframe
    df = df.copy()
    df['Cluster'] = clusters
    
    # Convert SubmissionDate to datetime
    df['SubmissionDate'] = pd.to_datetime(df['SubmissionDate'])
    
    # Extract year and month for analysis
    df['Year'] = df['SubmissionDate'].dt.year
    df['Month'] = df['SubmissionDate'].dt.month
    df['YearMonth'] = df['SubmissionDate'].dt.to_period('M')
    
    print("=" * 80)
    print("TEMPORAL TREND ANALYSIS")
    print("=" * 80)
    
    # Basic temporal statistics
    year_range = df['Year'].unique()
    print(f"\n1. TEMPORAL OVERVIEW:")
    print(f"   • Data spans from {df['SubmissionDate'].min().strftime('%Y-%m-%d')} to {df['SubmissionDate'].max().strftime('%Y-%m-%d')}")
    print(f"   • Years covered: {sorted(year_range)}")
    print(f"   • Total months: {df['YearMonth'].nunique()}")
    
    return df

def plot_temporal_distribution(df, keywords_df):
    """
    Create plots showing cluster distribution over time:
    - Stacked bar chart of cluster percentage by year.
    - Line plot of yearly trends for top clusters.
    - Heatmap of cluster distribution by year.
    - Line plot of seasonal (monthly) patterns.
    """
    # Filter out noise points for some analyses
    meaningful_df = df[df['Cluster'] != -1]
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Temporal Distribution of Clusters', fontsize=16, fontweight='bold')
    
    # Get cluster labels from keywords
    cluster_labels = {}
    for _, row in keywords_df.iterrows():
        cluster_id = row['cluster']
        if cluster_id != -1:
            top_keywords = ', '.join(row['Yake Short'].split(', ')[:2])
            cluster_labels[cluster_id] = f"{cluster_id}: {top_keywords}"

    # Plot 1: Cluster frequency by year (stacked bar chart)
    cluster_year = pd.crosstab(df['Year'], df['Cluster'])
    cluster_year_pct = cluster_year.div(cluster_year.sum(axis=1), axis=0) * 100
    
    colors = plt.cm.tab20(np.linspace(0, 1, len(cluster_year_pct.columns)))
    cluster_year_pct.plot(kind='bar', stacked=True, ax=axes[0, 0], 
                          color=[colors[i % len(colors)] for i in range(len(cluster_year_pct.columns))],
                          alpha=0.8)
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Percentage of Reports (%)')
    axes[0, 0].set_title('Cluster Distribution by Year (Percentage)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Plot 2: YEARLY trend lines for top clusters
    yearly_trends = df.groupby(['Year', 'Cluster']).size().unstack(fill_value=0)
    top_clusters = meaningful_df['Cluster'].value_counts().head(6).index
    
    for cluster in top_clusters:
        if cluster in yearly_trends.columns:
            axes[0, 1].plot(yearly_trends.index, yearly_trends[cluster], 
                            marker='o', markersize=6, linewidth=2.5, 
                            label=cluster_labels.get(cluster, f'Cluster {cluster}'))
    
    axes[0, 1].set_xlabel('Year')
    axes[0, 1].set_ylabel('Number of Reports')
    axes[0, 1].set_title('Yearly Trends for Top Clusters')
    axes[0, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Heatmap of cluster distribution by year
    cluster_year_normalized = cluster_year.div(cluster_year.sum(axis=0), axis=1) * 100
    
    im = axes[1, 0].imshow(cluster_year_normalized.T, cmap='YlOrRd', aspect='auto', alpha=0.8)
    axes[1, 0].set_xlabel('Year')
    axes[1, 0].set_ylabel('Cluster')
    axes[1, 0].set_title('Cluster Distribution Heatmap (Column Normalized)')
    
    years = sorted(df['Year'].unique())
    clusters = sorted([c for c in cluster_year_normalized.columns if c != -1])
    axes[1, 0].set_xticks(range(len(years)))
    axes[1, 0].set_xticklabels(years)
    axes[1, 0].set_yticks(range(len(clusters)))
    axes[1, 0].set_yticklabels(clusters)
    
    plt.colorbar(im, ax=axes[1, 0], label='Percentage (%)')
    
    # Plot 4: Seasonal patterns (monthly distribution)
    monthly_patterns = df.groupby(['Month', 'Cluster']).size().unstack(fill_value=0)
    monthly_patterns_pct = monthly_patterns.div(monthly_patterns.sum(axis=1), axis=0) * 100
    
    for cluster in top_clusters[:4]: # Top 4 for clarity
        if cluster in monthly_patterns_pct.columns:
            axes[1, 1].plot(monthly_patterns_pct.index, monthly_patterns_pct[cluster], 
                            marker='s', markersize=4, linewidth=2,
                            label=cluster_labels.get(cluster, f'Cluster {cluster}'))
    
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Percentage of Monthly Reports (%)')
    axes[1, 1].set_title('Seasonal Patterns by Cluster')
    axes[1, 1].set_xticks(range(1, 13))
    axes[1, 1].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig

def create_temporal_summary_statistics(df, keywords_df):
    """
    Create detailed statistical summary of temporal patterns for each cluster.
    """
    meaningful_df = df[df['Cluster'] != -1]
    
    print("\n2. TEMPORAL PATTERNS BY CLUSTER:")
    
    # Cluster emergence and persistence
    cluster_first_occurrence = meaningful_df.groupby('Cluster')['Year'].min()
    cluster_last_occurrence = meaningful_df.groupby('Cluster')['Year'].max()
    cluster_years_active = meaningful_df.groupby('Cluster')['Year'].nunique()
    
    temporal_stats = []
    for cluster in sorted(meaningful_df['Cluster'].unique()):
        cluster_data = meaningful_df[meaningful_df['Cluster'] == cluster]
        first_year = cluster_first_occurrence[cluster]
        last_year = cluster_last_occurrence[cluster]
        years_active = cluster_years_active[cluster]
        
        # Yearly growth rate calculation
        yearly_counts = cluster_data['Year'].value_counts().sort_index()
        growth_rate = 0
        if len(yearly_counts) > 1 and yearly_counts.iloc[0] > 0:
            growth_rate = (yearly_counts.iloc[-1] - yearly_counts.iloc[0]) / yearly_counts.iloc[0] * 100
        elif len(yearly_counts) > 1 and yearly_counts.iloc[0] == 0:
            growth_rate = np.inf if yearly_counts.iloc[-1] > 0 else 0
        
        # Get top keywords
        keywords_row = keywords_df[keywords_df['cluster'] == cluster]
        top_keywords = ', '.join(keywords_row.iloc[0]['Yake Short'].split(', ')[:3]) if not keywords_row.empty else "N/A"
        
        temporal_stats.append({
            'Cluster': cluster,
            'First Year': first_year,
            'Last Year': last_year,
            'Years Active': years_active,
            'Total Reports': len(cluster_data),
            'Growth Rate %': f"{growth_rate:+.1f}%" if growth_rate != np.inf else "+inf%",
            'Top Keywords': top_keywords
        })
    
    temporal_df = pd.DataFrame(temporal_stats)
    print(temporal_df.to_string(index=False, max_colwidth=25))
    
    # Yearly trends analysis
    print(f"\n3. YEARLY TREND ANALYSIS:")
    yearly_totals = df['Year'].value_counts().sort_index()
    print("Yearly report counts:")
    for year, count in yearly_totals.items():
        meaningful_count = len(meaningful_df[meaningful_df['Year'] == year])
        noise_count = count - meaningful_count
        print(f"   {year}: {count} total ({meaningful_count} clustered, {noise_count} noise)")
    
    return temporal_df

def plot_cluster_growth_trends(df, keywords_df):
    """
    Plot cumulative growth of all reports and the emergence timeline of clusters.
    """
    # Sort by date
    df_sorted = df.sort_values('SubmissionDate')
    
    # Calculate cumulative counts
    df_sorted['CumulativeCount'] = range(1, len(df_sorted) + 1)
    
    # Create cumulative cluster appearance
    cluster_first_appearance = df_sorted.groupby('Cluster').first()['CumulativeCount']
    
    # Plot cumulative growth
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Cumulative reports over time
    ax1.plot(df_sorted['SubmissionDate'], df_sorted['CumulativeCount'], 
             linewidth=2, color='navy', alpha=0.8)
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Cumulative Number of Reports')
    ax1.set_title('Cumulative Growth of Reports')
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Plot 2: Cluster emergence timeline
    meaningful_clusters = [c for c in cluster_first_appearance.index if c != -1]
    cluster_labels = {}
    for cluster in meaningful_clusters:
        keywords_row = keywords_df[keywords_df['cluster'] == cluster]
        if not keywords_row.empty:
            label = ', '.join(keywords_row.iloc[0]['Yake Short'].split(', ')[:2])
            cluster_labels[cluster] = f"{cluster}: {label}"
    
    for i, cluster in enumerate(meaningful_clusters):
        first_date = df_sorted[df_sorted['Cluster'] == cluster].iloc[0]['SubmissionDate']
        
        ax2.scatter(first_date, i, s=100, alpha=0.7, 
                    label=cluster_labels.get(cluster, f'Cluster {cluster}'))
    
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Cluster ID')
    ax2.set_title('Cluster First Appearance Timeline')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return fig

def plot_yearly_comparison(df, keywords_df):
    """
    Create additional yearly comparison plots for volume, diversity, and cluster dominance.
    """
    meaningful_df = df[df['Cluster'] != -1]
    
    # Calculate yearly statistics
    yearly_stats = meaningful_df.groupby('Year').agg({
        'Cluster': ['count', 'nunique']
    }).round(2)
    yearly_stats.columns = ['Total Reports', 'Cluster Diversity']
    
    # Yearly cluster composition
    yearly_cluster_pct = meaningful_df.groupby(['Year', 'Cluster']).size().unstack(fill_value=0)
    yearly_cluster_pct = yearly_cluster_pct.div(yearly_cluster_pct.sum(axis=1), axis=0) * 100
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Yearly diversity and volume
    ax1.bar(yearly_stats.index, yearly_stats['Total Reports'], 
            alpha=0.7, color='skyblue', label='Total Reports')
    ax1_line = ax1.twinx()
    ax1_line.plot(yearly_stats.index, yearly_stats['Cluster Diversity'], 
                  color='red', marker='o', linewidth=2, label='Cluster Diversity')
    
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Number of Reports', color='skyblue')
    ax1_line.set_ylabel('Number of Clusters', color='red')
    ax1.set_title('Yearly Report Volume vs Cluster Diversity')
    ax1.tick_params(axis='x', rotation=45)
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax1_line.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    # Plot 2: Yearly dominance of top clusters
    top_clusters = meaningful_df['Cluster'].value_counts().head(3).index
    
    for cluster in top_clusters:
        if cluster in yearly_cluster_pct.columns:
            ax2.plot(yearly_cluster_pct.index, yearly_cluster_pct[cluster], 
                     marker='s', linewidth=2, markersize=6,
                     label=f'Cluster {cluster}')
    
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Percentage of Yearly Reports (%)')
    ax2.set_title('Yearly Dominance of Top Clusters')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return fig

def analyze_cluster_correlations(df):
    """
    Analyze correlations between cluster frequencies over time (yearly).
    """
    # YEARLY cluster frequencies
    yearly_clusters = df.groupby(['Year', 'Cluster']).size().unstack(fill_value=0)
    
    # Calculate correlation matrix, excluding noise cluster -1 if present
    meaningful_clusters_freq = yearly_clusters.drop(columns=[-1], errors='ignore')
    correlation_matrix = meaningful_clusters_freq.corr()
    
    # Plot correlation heatmap
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Mask upper triangle
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    
    # Create heatmap
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', 
                center=0, ax=ax, square=True, fmt='.2f')
    ax.set_title('Yearly Cluster Frequency Correlations')
    
    plt.tight_layout()
    plt.show()
    
    # Identify strongly correlated clusters
    strong_correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr = correlation_matrix.iloc[i, j]
            if abs(corr) > 0.5: # Threshold for strong correlation
                strong_correlations.append({
                    'Cluster_A': correlation_matrix.columns[i],
                    'Cluster_B': correlation_matrix.columns[j],
                    'Correlation': corr
                })
    
    if strong_correlations:
        print(f"\n4. STRONG YEARLY CORRELATIONS (|r| > 0.5):")
        corr_df = pd.DataFrame(strong_correlations)
        print(corr_df.to_string(index=False, float_format='%.3f'))
    
    return correlation_matrix

# Main execution for Temporal Analysis
if __name__ == "__main__":
    # ASSUME 'results' DICTIONARY IS AVAILABLE HERE
    # results = your_clustering_output 
    
    # Example placeholder for results (you must replace this with your actual data source)
    try:
        results = results 
    except NameError:
        print("Error: 'results' dictionary not found. Please ensure it is defined and contains 'dataframe', 'clustering', and 'keywords' keys.")
        exit()
        
    print("\n" + "=" * 80)
    print("TEMPORAL ANALYSIS EXECUTION")
    print("=" * 80)
    
    # 1. Temporal analysis and preprocessing
    df_with_clusters = analyze_temporal_trends(results)
    
    # 2. Temporal visualizations
    print("\nGenerating temporal distribution plots...")
    temporal_fig = plot_temporal_distribution(df_with_clusters, results['keywords'])
    
    # 3. Additional yearly comparison
    print("\nGenerating yearly comparison plots...")
    yearly_fig = plot_yearly_comparison(df_with_clusters, results['keywords'])
    
    # 4. Temporal statistics
    temporal_stats = create_temporal_summary_statistics(df_with_clusters, results['keywords'])
    
    # 5. Growth trends
    print("\nGenerating growth trend plots...")
    growth_fig = plot_cluster_growth_trends(df_with_clusters, results['keywords'])
    
    # 6. Correlation analysis
    print("\nAnalyzing yearly correlations...")
    correlation_matrix = analyze_cluster_correlations(df_with_clusters)
    
    # 7. Final temporal insights
    print("\n" + "=" * 80)
    print("YEARLY TREND INSIGHTS")
    print("=" * 80)
    
    # Calculate some key insights
    yearly_cluster_diversity = df_with_clusters.groupby('Year')['Cluster'].nunique()
    most_diverse_year = yearly_cluster_diversity.idxmax()
    least_diverse_year = yearly_cluster_diversity.idxmin()
    
    # Yearly growth rates for overall reports
    yearly_totals = df_with_clusters['Year'].value_counts().sort_index()
    overall_growth = 0
    if len(yearly_totals) > 1 and yearly_totals.iloc[0] > 0:
        overall_growth = (yearly_totals.iloc[-1] - yearly_totals.iloc[0]) / yearly_totals.iloc[0] * 100
    
    print(f"• Year with most cluster diversity: {most_diverse_year} ({yearly_cluster_diversity[most_diverse_year]} clusters)")
    print(f"• Year with least cluster diversity: {least_diverse_year} ({yearly_cluster_diversity[least_diverse_year]} clusters)")
    print(f"• Overall growth in reports (start to end year): {overall_growth:+.1f}%")
    
    # Identify emerging clusters 
    recent_year = df_with_clusters['Year'].max()
    if recent_year > df_with_clusters['Year'].min():
        previous_year = recent_year - 1
        recent_clusters = set(df_with_clusters[df_with_clusters['Year'] == recent_year]['Cluster'].unique())
        previous_clusters = set(df_with_clusters[df_with_clusters['Year'] == previous_year]['Cluster'].unique())
        new_clusters = (recent_clusters - previous_clusters) - {-1} 
        
        if new_clusters:
            # Look up keywords for new clusters
            new_clusters_labels = []
            keywords_df = results['keywords']
            for c in sorted(list(new_clusters)):
                keywords_row = keywords_df[keywords_df['cluster'] == c]
                if not keywords_row.empty:
                    top_keyword = keywords_row.iloc[0]['Yake Short'].split(', ')[0]
                    new_clusters_labels.append(f"{c}: {top_keyword}")
                else:
                    new_clusters_labels.append(f"{c}")
            
            print(f"• New clusters emerging in {recent_year}: {new_clusters_labels}")
    
    # Most consistent clusters (appear in all years where data exists)
    years_present = df_with_clusters['Year'].nunique()
    cluster_persistence = df_with_clusters.groupby('Cluster')['Year'].nunique()
    most_persistent = cluster_persistence[cluster_persistence == years_present].index.tolist()
    print(f"• Most persistent clusters (appear in all {years_present} years): {most_persistent}")